In [ ]:
import itertools
import numpy as np

# ==== パラメータ設定 ====
num_orbitals = 6
num_particles = 3
delta = 1.0   # 1体項の等間隔エネルギー
v = 0.3       # 2体相互作用（同じ縮退ペア間のみ）
w3 = 0.2      # 3体相互作用（全てのトリプレット間）

# ==== 基底生成 ====
def generate_basis(num_orbitals, num_particles):
    basis = []
    for bits in itertools.combinations(range(num_orbitals), num_particles):
        occ = np.zeros(num_orbitals, dtype=int)
        occ[list(bits)] = 1
        basis.append(tuple(occ))
    return basis

basis = generate_basis(num_orbitals, num_particles)
dim = len(basis)

# ==== 生成消滅演算の定義（フェルミ符号を含む） ====
def apply_annihilate(state, i):
    state = list(state)
    if state[i] == 0:
        return None, 0
    sign = (-1)**sum(state[:i])
    state[i] = 0
    return tuple(state), sign

def apply_create(state, i):
    state = list(state)
    if state[i] == 1:
        return None, 0
    sign = (-1)**sum(state[:i])
    state[i] = 1
    return tuple(state), sign

def apply_cdag_c(state, i, j):
    """c_i† c_j"""
    s1, sign1 = apply_annihilate(state, j)
    if s1 is None:
        return None, 0
    s2, sign2 = apply_create(s1, i)
    if s2 is None:
        return None, 0
    return s2, sign1 * sign2

# ==== 1体ハミルトニアン ====
H1 = np.zeros((dim, dim))
for p, s in enumerate(basis):
    for i, occ in enumerate(s):
        if occ == 1:
            energy = delta * (i // 2)  # 各2縮退レベルごとに等間隔
            H1[p, p] += energy

#==== 2体ハミルトニアン（改訂版）====
pairs = [(0, 1), (2, 3), (4, 5)]
H2 = np.zeros((dim, dim))

def find_pair_and_single(state):
    """3粒子状態から (pair_index, single_orbital) を特定"""
    for p_idx, (a, b) in enumerate(pairs):
        if state[a] == 1 and state[b] == 1:
            s = [i for i in range(len(state)) if state[i] == 1 and i not in (a, b)][0]
            return p_idx, s
    return None, None

for i, bra in enumerate(basis):
    p_bra, s_bra = find_pair_and_single(bra)
    if p_bra is None:
        continue  # ペアがない状態は作用しない

    for j, ket in enumerate(basis):
        p_ket, s_ket = find_pair_and_single(ket)
        if p_ket is None:
            continue

        # ペアと単粒子の両方が遷移できる
        # 全結合を許して同一強度v
        H2[i, j] += v 

# ==== 3体ハミルトニアン ====
triplets = [(0,1,2), (1,2,3), (2,3,4), (3,4,5)]
H3 = np.zeros((dim, dim))

# 全ての triplet → triplet 結合を入れる
for T1 in triplets:
    for T2 in triplets:
        for i, s in enumerate(basis):
            # T2をすべて占有している場合のみ作用
            if all(s[k] == 1 for k in T2):
                s_new = list(s)
                sign = 1
                # まず T2 をアニヒレート
                for k in reversed(T2):
                    s_new, sgn = apply_annihilate(s_new, k)
                    if s_new is None:
                        break
                    sign *= sgn
                if s_new is None:
                    continue
                # 次に T1 を作る
                for k in T1:
                    s_new, sgn = apply_create(s_new, k)
                    if s_new is None:
                        break
                    sign *= sgn
                if s_new is None:
                    continue
                s_new = tuple(s_new)
                if s_new in basis:
                    j = basis.index(s_new)
                    H3[i, j] += w3 * sign

# ==== 総ハミルトニアン ====
H = H1 + H2 + H3
H = (H + H.conj())/2# Hermitize（実対称に）

# ==== 結果表示 ====
np.set_printoptions(precision=2, suppress=True)
print("Hamiltonian matrix (dim={}):".format(dim))
print(H)

# ==== 固有値 ====
eigvals, _ = np.linalg.eigh(H)
print("\nEigenvalues:")
print(np.round(eigvals, 4))

Hamiltonian matrix (dim=20):
[[1.5 0.3 0.3 0.3 0.3 0.  0.  0.  0.  0.3 0.5 0.  0.  0.  0.  0.3 0.5 0.3
  0.3 0.5]
 [0.3 1.3 0.3 0.3 0.3 0.  0.  0.  0.  0.3 0.3 0.  0.  0.  0.  0.3 0.3 0.3
  0.3 0.3]
 [0.3 0.3 2.3 0.3 0.3 0.  0.  0.  0.  0.3 0.3 0.  0.  0.  0.  0.3 0.3 0.3
  0.3 0.3]
 [0.3 0.3 0.3 2.3 0.3 0.  0.  0.  0.  0.3 0.3 0.  0.  0.  0.  0.3 0.3 0.3
  0.3 0.3]
 [0.3 0.3 0.3 0.3 2.3 0.  0.  0.  0.  0.3 0.3 0.  0.  0.  0.  0.3 0.3 0.3
  0.3 0.3]
 [0.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. ]
 [0.  0.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. ]
 [0.3 0.3 0.3 0.3 0.3 0.  0.  0.  0.  4.3 0.3 0.  0.  0.  0.  0.3 0.3 0.3
  0.3 0.3]
 [0.5 0.3 0.3 0.3 0.3 0.  0.  0.  0.  0.3 2.5 0.  0.  0.  0.  0.3 0.5 0.3
  0.3 0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

In [6]:
import itertools
import numpy as np

# ==== 全基底（64次元） ====
all_basis = [tuple(int(x) for x in format(i, '06b')) for i in range(2**6)]

# ==== 3粒子空間の基底インデックスを求める ====
basis_index = [all_basis.index(b) for b in basis]

# ==== 20次元→64次元への埋め込み ====
H_full = np.zeros((64, 64), dtype=complex)
for i, bi in enumerate(basis_index):
    for j, bj in enumerate(basis_index):
        H_full[bi, bj] = H[i, j]

# ==== パウリ行列定義 ====
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
paulis = [I, X, Y, Z]
labels = ['I','X','Y','Z']

# ==== Pauli 展開 ====
coeffs = {}
for ops in itertools.product(range(4), repeat=6):
    P = paulis[ops[0]]
    for k in ops[1:]:
        P = np.kron(P, paulis[k])
    c = np.trace(P @ H_full) / 64
    if abs(c) > 1e-8:
        label = ''.join(labels[k] for k in ops)
        coeffs[label] = np.real_if_close(c)

# ==== 出力 ====
print(f"\nNonzero Pauli terms ({len(coeffs)} terms):")
for k, v in sorted(coeffs.items(), key=lambda x: -abs(x[1])):
    print(f"{k:10s} : {v:.6f}")



Nonzero Pauli terms (652 terms):
IIIIII     : 1.006250
ZZZZZZ     : -1.006250
IIZIIZ     : -0.218750
IZIIZI     : -0.218750
IZZIZZ     : 0.218750
ZIIZII     : -0.218750
ZIZZIZ     : 0.218750
ZZIZZI     : 0.218750
IIZIZI     : -0.212500
IZIIIZ     : -0.212500
ZIZZZI     : 0.212500
ZZIZIZ     : 0.212500
IZIZII     : -0.212500
ZIZIZZ     : 0.212500
IZZZIZ     : 0.212500
ZIIIZI     : -0.212500
IZZZZI     : 0.206250
ZIIIIZ     : -0.206250
IZIZZZ     : 0.206250
ZIZIII     : -0.206250
IIIZIZ     : -0.206250
ZZZIZI     : 0.206250
IZZIII     : -0.200000
ZIIZZZ     : 0.200000
IIIZZI     : -0.200000
ZZZIIZ     : 0.200000
IZZZZZ     : -0.193750
ZIIIII     : 0.193750
IIIIZI     : -0.187500
IZIIII     : 0.187500
ZIZZZZ     : -0.187500
ZZZZIZ     : 0.187500
IIIIIZ     : -0.181250
ZZZZZI     : 0.181250
IIZZII     : -0.168750
ZZIIZZ     : 0.168750
IIIIZZ     : -0.162500
ZZZZII     : 0.162500
IIZZZZ     : 0.162500
ZZIIII     : -0.162500
IIZIZZ     : 0.125000
ZZIZII     : -0.125000
IIIZZZ     : 0.125000

In [7]:
import itertools
import numpy as np
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper

# --- 前のHを使う ---
# Hは20×20の行列
dim = H.shape[0]
N_orb = 6  # 軌道数

# --- Basis 生成（3粒子/6軌道） ---
def generate_basis(num_orbitals, num_particles):
    basis = []
    for bits in itertools.combinations(range(num_orbitals), num_particles):
        occ = np.zeros(num_orbitals, dtype=int)
        occ[list(bits)] = 1
        basis.append(tuple(occ))
    return basis

basis = generate_basis(N_orb, 3)

# --- FermionicOp用に辞書を作成 ---
terms = {}
for i, bra in enumerate(basis):
    for j, ket in enumerate(basis):
        coeff = H[i, j]
        if abs(coeff) < 1e-12:
            continue

        ops = []
        for k in range(N_orb):
            if bra[k] == 1 and ket[k] == 0:
                ops.append(f"+_{k}")
            elif bra[k] == 0 and ket[k] == 1:
                ops.append(f"-_{k}")

        label = ' '.join(ops) if ops else ""  # 恒等項は空文字
        terms[label] = terms.get(label, 0.0) + coeff

fermionic_ham = FermionicOp(terms, num_spin_orbitals=N_orb)


# --- FermionicOp 生成 ---
fermionic_ham = FermionicOp(terms, num_spin_orbitals=N_orb)

# --- Jordan-Wigner変換 ---
mapper = JordanWignerMapper()
qubit_ham = mapper.map(fermionic_ham)

# --- 結果表示 ---
print("=== Fermionic Hamiltonian ===")
print(fermionic_ham)

print("\n=== Pauli-encoded Hamiltonian ===")
print(qubit_ham)



=== Fermionic Hamiltonian ===
Fermionic Operator
number spin orbitals=6, number terms=73
  64.39999999999999
+ 0.6 * ( +_2 -_3 )
+ 0.6 * ( +_2 -_4 )
+ 0.8 * ( +_2 -_5 )
+ 0.6 * ( +_1 -_3 )
+ 0.8 * ( +_1 +_2 -_4 -_5 )
+ 0.8 * ( +_0 -_3 )
+ 0.6 * ( +_0 +_2 -_4 -_5 )
+ 0.8 * ( +_0 +_1 -_3 -_4 )
+ 0.6 * ( +_0 +_1 -_3 -_5 )
+ 0.6 * ( +_0 +_1 -_4 -_5 )
+ 0.5 * ( +_0 +_1 +_2 -_3 -_4 -_5 )
+ 0.6 * ( -_2 +_3 )
+ 0.6 * ( +_3 -_4 )
+ 0.6 * ( +_3 -_5 )
+ 0.6 * ( +_1 -_2 )
+ 0.6 * ( +_1 +_3 -_4 -_5 )
+ 0.6 * ( +_0 -_2 )
+ 0.6 * ( +_0 +_3 -_4 -_5 )
+ 0.6 * ( +_0 +_1 -_2 -_4 )
+ 0.6 * ( +_0 +_1 -_2 -_5 )
+ 0.3 * ( +_0 +_1 -_2 +_3 -_4 -_5 )
+ 0.6 * ( -_2 +_4 )
+ 0.6 * ( -_3 +_4 )
+ 0.6 * ( +_4 -_5 )
+ 0.6 * ( +_1 -_2 -_3 +_4 )
+ 0.6 * ( +_1 -_5 )
+ 0.6 * ( +_0 -_2 -_3 +_4 )
+ 0.6 * ( +_0 -_5 )
+ 0.6 * ( +_0 +_1 -_2 -_3 )
+ 0.3 * ( +_0 +_1 -_2 -_3 +_4 -_5 )
+ 0.8 * ( -_2 +_5 )
+ 0.6 * ( -_3 +_5 )
+ 0.6 * ( -_4 +_5 )
+ 0.6 * ( +_1 -_2 -_3 +_5 )
+ 0.8 * ( +_1 -_4 )
+ 0.6 * ( +_0 -_2 -_3 +_5 )
+ 0.6 * ( +

In [8]:
import numpy as np

# qubit_ham は Jordan-Wigner 変換後の SparsePauliOp
# 量子ビット数
n_qubits = 6

# 数値行列に変換
H_matrix = qubit_ham.to_matrix()

# 確認
np.set_printoptions(precision=3, suppress=True)
print("Pauli変換後の行列 (dim={}):".format(H_matrix.shape[0]))
print(H_matrix)

# 固有値も計算可能
eigvals = np.linalg.eigvalsh(H_matrix)
print("\nEigenvalues:")
print(np.round(eigvals, 4))

Pauli変換後の行列 (dim=64):
[[64.4+0.j  0. +0.j  0. +0.j ...  0. +0.j  0. +0.j -0. +0.j]
 [ 0. +0.j 64.4+0.j  0.6+0.j ...  0. +0.j -0. +0.j  0. +0.j]
 [ 0. +0.j -0.6+0.j 64.4+0.j ...  0. +0.j  0. +0.j  0. +0.j]
 ...
 [ 0. +0.j  0. +0.j -0. +0.j ... 64.4+0.j  0.6+0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j ... -0.6+0.j 64.4+0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j ...  0. +0.j  0. +0.j 64.4+0.j]]

Eigenvalues:
[60.806 60.861 60.88  61.2   61.467 61.562 61.593 61.757 62.15  62.225
 62.415 62.42  62.504 62.557 62.628 62.743 62.958 63.064 63.2   63.2
 63.2   63.212 63.413 63.425 63.434 63.6   63.6   63.6   64.    64.
 64.199 64.4   64.4   64.8   64.8   64.814 65.114 65.2   65.2   65.2
 65.519 65.556 65.59  65.6   65.6   65.6   65.848 65.863 65.889 66.02
 66.109 66.193 66.389 66.515 66.594 66.863 66.99  67.075 67.187 67.6
 67.714 67.773 67.85  67.863]
